## Assessing a set of additive utility functions for multicriteria decision-making, the UTA method.

### Implementação do artigo publicado por E.JACQUET-LAGREZE e J.SISKOS

#### Daniel de Araujo Pereira

#### Problema de Progamação Linear

$$ [min] F = \sum \sigma(a) $$

sujeito a:

$ \sum [ u_i[g_i(a)] - u_i[g_i(b)] ] + \sigma(a) - \sigma(b) >= \delta $ se $ aPb $
<br>
$ \sum [ u_i[g_i(a)] - u_i[g_i(b)] ] + \sigma(a) - \sigma(b) = 0 $ se $ aIb $
<br>
$ u_i(g_i^{j+1}) - u_i(g_i^j) >= s_i $ para todo $ i $ e $ j $
<br>
$ \sum u_i(g_i^{*}) = 1 $
<br>
$ u_i(g_{i*}) = 0, u_i(g_i^j) >= 0, \sigma(a) >= 0 $ para todo $i$ e $j$ e para todo $ a \in A' $

In [1]:
# creating the dataset in the article

import pandas as pd
from pandas import read_csv

my_data = read_csv('cars.csv', encoding = 'ISO-8859-1')
my_data.head(10)

,car,rank,max_speed,consumption_town,consumption_road,horse_power,space,price
0,Peugeot 505 GR,1,173,11.4,10.01,10,7.88,49500
1,Opel Record 2000 LS,2,176,12.3,10.48,11,7.96,46700
2,Citroen Visa Super E,3,142,8.2,7.30,5,5.65,32100
3,VW Golf 1300 GLS,4,148,10.5,9.61,7,6.15,39150
4,Citroen CX 2400 Pallas,5,178,14.5,11.05,13,8.06,64700
5,Mercedes 230,6,180,13.6,10.40,13,8.47,75700
6,BMW 520,7,182,12.7,12.26,11,7.81,68593
7,Volvo 244 DL,8,145,14.3,12.95,11,8.38,55000
8,Peugeot 104 ZS,9,161,8.6,8.42,7,5.11,35200
9,Citroen Dyane,10,117,7.2,6.75,3,5.81,24800


In [2]:
# defining the restrictions and variables

delta = 0.01

my_g = read_csv('criteria.csv', encoding = 'ISO-8859-1')

In [3]:
def restriction1_criteria(G_row, criteria):
    """ 
    This returns a dictionary U with u_criteria_gi: i_value and 
    u_criteria_gj: j_value that will be a row for the third set of restrictions
    
    """
    U = pd.DataFrame()
    gmin = G_row['g_i*']
    gmax = G_row['g_i^*']
    step = (gmax - gmin)/(G_row['a_i'] - 1)
    for car in range(len(criteria.index)):
        gmin = G_row['g_i*']
        for i in range(G_row['a_i'] - 1):
            g = gmin + step
            #print ("range: %s to %s" % (gmin, g))
            if gmin <= criteria.loc[car] <= g:
                coef = (criteria.loc[car] - gmin)/(g - gmin)
                i_name = 'u_'+ G_row.name + '_' + "{:.1f}".format(gmin)
                j_name = 'u_'+ G_row.name + '_' + "{:.1f}".format(g)
                i_value = 1 - coef
                j_value = coef
                U = U.append({i_name: i_value, j_name: j_value}, ignore_index=True)
            gmin = gmin + step
    #print(U.head())
    return U

def restriction1(G, data):
    new_data = data
    for criteria in G.index.tolist():
        new_data = pd.concat([new_data, restriction1_criteria(G.loc[criteria], data[criteria])], axis=1)
        #print(new_data.index)
    return new_data

def add_sigma(data):
    U = pd.DataFrame()
    for car in range(len(data.index)):
        i_name = 'sigma_'+ str(car)
        i_value = 1
        U = U.append({i_name: i_value}, ignore_index=True)
    new_data = pd.concat([data, U], axis=1)
    return new_data

In [4]:
#restrições
my_g = my_g.rename({0: 'max_speed',
                    1: 'consumption_town',
                    2: 'consumption_road',
                    3: 'horse_power',
                    4: 'space',
                    5: 'price'
                   })
my_g.head()

,g_i*,g_i^*,a_i,s_i
max_speed,110,190,5,0
consumption_town,-15,-7,4,0
consumption_road,-13,-6,4,0
horse_power,3,13,5,0
space,5,9,4,0


In [5]:
U = restriction1(my_g, my_data)

In [6]:
U.head(12)

,car,rank,max_speed,consumption_town,consumption_road,horse_power,space,price,u_max_speed_170.0,u_max_speed_190.0,...,u_max_speed_110.0,u_horse_power_10.5,u_horse_power_8.0,u_horse_power_13.0,u_horse_power_3.0,u_horse_power_5.5,u_space_7.7,u_space_9.0,u_space_5.0,u_space_6.3
0,Peugeot 505 GR,1,173,11.4,10.01,10,7.88,49500,0.85,0.15,...,NaN,0.8,0.2,NaN,NaN,NaN,0.8400,0.1600,NaN,NaN
1,Opel Record 2000 LS,2,176,12.3,10.48,11,7.96,46700,0.70,0.30,...,NaN,0.8,NaN,0.2,NaN,NaN,0.7800,0.2200,NaN,NaN
2,Citroen Visa Super E,3,142,8.2,7.30,5,5.65,32100,NaN,NaN,...,NaN,NaN,NaN,NaN,0.2,0.8,NaN,NaN,0.5125,0.4875
3,VW Golf 1300 GLS,4,148,10.5,9.61,7,6.15,39150,NaN,NaN,...,NaN,NaN,0.6,NaN,NaN,0.4,NaN,NaN,0.1375,0.8625
4,Citroen CX 2400 Pallas,5,178,14.5,11.05,13,8.06,64700,0.60,0.40,...,NaN,0.0,NaN,1.0,NaN,NaN,0.7050,0.2950,NaN,NaN
5,Mercedes 230,6,180,13.6,10.40,13,8.47,75700,0.50,0.50,...,NaN,0.0,NaN,1.0,NaN,NaN,0.3975,0.6025,NaN,NaN
6,BMW 520,7,182,12.7,12.26,11,7.81,68593,0.40,0.60,...,NaN,0.8,NaN,0.2,NaN,NaN,0.8925,0.1075,NaN,NaN
7,Volvo 244 DL,8,145,14.3,12.95,11,8.38,55000,NaN,NaN,...,NaN,0.8,NaN,0.2,NaN,NaN,0.4650,0.5350,NaN,NaN
8,Peugeot 104 ZS,9,161,8.6,8.42,7,5.11,35200,0.55,NaN,...,NaN,NaN,0.6,NaN,NaN,0.4,NaN,NaN,0.9175,0.0825
9,Citroen Dyane,10,117,7.2,6.75,3,5.81,24800,NaN,NaN,...,0.65,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.3925,0.6075


In [7]:
restriction1 = add_sigma(U)

In [8]:
restriction1 = restriction1.ix[:, 'u_max_speed_170.0':]
restriction1 = restriction1.fillna(0)

In [9]:
restriction1.head(10)

,u_max_speed_170.0,u_max_speed_190.0,u_max_speed_130.0,u_max_speed_150.0,u_max_speed_110.0,u_horse_power_10.5,u_horse_power_8.0,u_horse_power_13.0,u_horse_power_3.0,u_horse_power_5.5,...,sigma_0,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5,sigma_6,sigma_7,sigma_8,sigma_9
0,0.85,0.15,0.00,0.00,0.00,0.8,0.2,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.70,0.30,0.00,0.00,0.00,0.8,0.0,0.2,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.00,0.40,0.60,0.00,0.0,0.0,0.0,0.2,0.8,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.00,0.10,0.90,0.00,0.0,0.6,0.0,0.0,0.4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.60,0.40,0.00,0.00,0.00,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.50,0.50,0.00,0.00,0.00,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.40,0.60,0.00,0.00,0.00,0.8,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.00,0.00,0.25,0.75,0.00,0.8,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.55,0.00,0.00,0.45,0.00,0.0,0.6,0.0,0.0,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.00,0.00,0.35,0.00,0.65,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
def subtract_rows(data):
    U = pd.DataFrame()
    for car in range(len(data.index)):
        for car2 in range((car+1), len(data.index)):
            U = U.append((data.loc[car] - data.loc[car2]) , ignore_index=True)
            U['b'] = delta
    return U.fillna(0)
                     
restriction1 = subtract_rows(restriction1)

In [11]:
restriction1.head(50)

,sigma_0,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5,sigma_6,sigma_7,sigma_8,sigma_9,...,u_max_speed_110.0,u_max_speed_130.0,u_max_speed_150.0,u_max_speed_170.0,u_max_speed_190.0,u_space_5.0,u_space_6.3,u_space_7.7,u_space_9.0,b
0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.15,-0.15,0.0000,0.0000,0.0600,-0.0600,0.01
1,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,-0.40,-0.60,0.85,0.15,-0.5125,-0.4875,0.8400,0.1600,0.01
2,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,-0.10,-0.90,0.85,0.15,-0.1375,-0.8625,0.8400,0.1600,0.01
3,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.25,-0.25,0.0000,0.0000,0.1350,-0.1350,0.01
4,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.35,-0.35,0.0000,0.0000,0.4425,-0.4425,0.01
5,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.45,-0.45,0.0000,0.0000,-0.0525,0.0525,0.01
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.00,-0.25,-0.75,0.85,0.15,0.0000,0.0000,0.3750,-0.3750,0.01
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.00,0.00,-0.45,0.30,0.15,-0.9175,-0.0825,0.8400,0.1600,0.01
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,-0.65,-0.35,0.00,0.85,0.15,-0.3925,-0.6075,0.8400,0.1600,0.01
9,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,-0.40,-0.60,0.70,0.30,-0.5125,-0.4875,0.7800,0.2200,0.01


In [12]:
# agora, a restrição 2!
# monotocidade (?)


def restriction2_criteria(G_row):

    U = pd.DataFrame()
    gmin = G_row['g_i*']
    gmax = G_row['g_i^*']
    s = G_row['s_i']
    step = (gmax - gmin)/(G_row['a_i'] - 1)
    for i in range(G_row['a_i'] - 1):
        g = gmax - step
        i_name = 'u_'+ G_row.name + '_' + "{:.1f}".format(gmax)
        j_name = 'u_'+ G_row.name + '_' + "{:.1f}".format(g)
        i_value = 1
        j_value = -1
        U = U.append({i_name: i_value, j_name: j_value, "b": s}, ignore_index=True)
        gmax = gmax - step
    #print(U.head(10))
    return U.fillna(0)

def restriction2(G, data):
    new_data = data
    for criteria in G.index.tolist():
        #print(criteria)
        new_data = new_data.append(restriction2_criteria(G.loc[criteria]), ignore_index=True)
        #print(new_data.index)
    return new_data.fillna(0)


restrictions = restriction1
restrictions = restriction2(my_g, restriction1)

In [13]:
def equal_restrictions(G, data):
    #since we need to have all the columns for the linprog() function to work
    #we will append the equality restritions to the upper restrictions and then take the last row
    #(the one we added and the one we really want)
    new_data = pd.DataFrame()
    for criteria in G.index.tolist():
        g_name = 'u_'+ G.loc[criteria].name + '_' + "{:.1f}".format(G.loc[criteria]['g_i^*'])
        g_value = 1
        #print(g_name)
        new_g = pd.DataFrame()
        new_g = new_g.append({g_name: g_value}, ignore_index=True)
        #print(new_g)
        new_data = pd.concat([new_data, new_g], axis=1)
        #print(new_data.index)
        
    new_data = data.append(new_data, ignore_index=True)
    
    return new_data.tail(1).fillna(0)

eqRestrictions = equal_restrictions(my_g, restrictions)

In [14]:
# get min names
min_names = []
for i in my_g.index.tolist():
    g_name = 'u_'+ my_g.loc[i].name + '_' + "{:.1f}".format(my_g.loc[i]['g_i*'])
    min_names.append(g_name)
min_names

['u_max_speed_110.0',
 'u_consumption_town_-15.0',
 'u_consumption_road_-13.0',
 'u_horse_power_3.0',
 'u_space_5.0',
 'u_price_-80000.0']

In [15]:
A_ub = restrictions.drop('b', 1)
b_ub = restrictions['b']

A_eq = eqRestrictions.drop('b', 1)
b_eq = eqRestrictions['b']

# create bounds for all variables
bnds = []
for i in list(A_ub.columns.values):
    if i in min_names:
        bnds.append((0,0))
    else:
        bnds.append((0,None))
        


In [16]:
# create cost function
cost = []
for i in list(A_ub.columns.values):
    if "sigma" in i:
        cost.append(1)
    else:
        cost.append(0)

In [17]:
from scipy.optimize import linprog

res = linprog(cost,
              A_ub=A_ub,
              b_ub=b_ub,
              A_eq=A_eq,
              b_eq=b_eq,
              bounds=bnds)


In [18]:
res

     fun: -0.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([ 0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,
        0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,
        0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,
        0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,
        0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,  0.01,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])
  status: 0
 success: True
       x: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [19]:
from IPython.display import display
pd.options.display.max_columns = None
display(b_ub)

0     0.01
1     0.01
2     0.01
3     0.01
4     0.01
5     0.01
6     0.01
7     0.01
8     0.01
9     0.01
10    0.01
11    0.01
12    0.01
13    0.01
14    0.01
15    0.01
16    0.01
17    0.01
18    0.01
19    0.01
20    0.01
21    0.01
22    0.01
23    0.01
24    0.01
25    0.01
26    0.01
27    0.01
28    0.01
29    0.01
      ... 
36    0.01
37    0.01
38    0.01
39    0.01
40    0.01
41    0.01
42    0.01
43    0.01
44    0.01
45    0.00
46    0.00
47    0.00
48    0.00
49    0.00
50    0.00
51    0.00
52    0.00
53    0.00
54    0.00
55    0.00
56    0.00
57    0.00
58    0.00
59    0.00
60    0.00
61    0.00
62    0.00
63    0.00
64    0.00
65    0.00
Name: b, dtype: float64